In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

24/06/09 14:11:20 WARN Utils: Your hostname, truongvude resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlp4s0)
24/06/09 14:11:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/09 14:11:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [50]:
df_fhv = spark.read.parquet("data/pq/fhv/*")

In [51]:
df_fhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01455|2019-10-02 16:10:49|2019-10-02 16:18:10|         264|         173|   NULL|                B01455|
|              B01559|2019-10-02 22:07:18|2019-10-02 22:30:18|         264|         264|   NULL|                B01559|
|              B01800|2019-10-04 13:16:00|2019-10-04 15:22:00|         264|         264|   NULL|                B01800|
|              B02613|2019-10-02 23:36:26|2019-10-02 23:36:27|         264|          39|   NULL|                B02613|
|              B01964|2019-10-02 20:26:00|2019-10-02 21:42:37|         132|         230|   NULL|                B01964|
|              B02245|2019-10-03 19:10:3

In [33]:
df_zones = spark.read \
    .option("header", "true") \
    .csv("taxi_zone_lookup.csv")

In [34]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [35]:
import pandas as pd

In [36]:
df_zones_pd = pd.read_csv("taxi_zone_lookup.csv", nrows = 1000)

In [37]:
spark.createDataFrame(df_zones_pd).schema

StructType([StructField('LocationID', LongType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [38]:
from pyspark.sql import types

In [39]:
zones_schema = types.StructType([
types.StructField('LocationID', types.IntegerType(), True), 
types.StructField('Borough', types.StringType(), True), 
types.StructField('Zone', types.StringType(), True), 
types.StructField('service_zone', types.StringType(), True)])

In [44]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(zones_schema) \
    .csv("taxi_zone_lookup.csv")

In [47]:
df_zones.write.parquet("data/pq/zones", mode="overwrite")

In [73]:
df_fhv = spark.read.parquet("data/pq/fhv/*")

In [72]:
df_zones = spark.read.parquet("data/pq/zones/*")

In [74]:
df_result = df_fhv.join(df_zones, df_fhv.PUlocationID == df_zones.LocationID)

In [75]:
df_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+-------+-----------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|LocationID|Borough|       Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+----------+-------+-----------+------------+
|              B01455|2019-10-02 16:10:49|2019-10-02 16:18:10|         264|         173|   NULL|                B01455|       264|Unknown|         NV|         N/A|
|              B01559|2019-10-02 22:07:18|2019-10-02 22:30:18|         264|         264|   NULL|                B01559|       264|Unknown|         NV|         N/A|
|              B01800|2019-10-04 13:16:00|2019-10-04 15:22:00|         264|         264|   NULL|                B01800|       264|Unknown|         NV|         N/A|
|              B

In [76]:
df_result.registerTempTable("result")

/home/truongvude/spark/spark-3.5.1-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [91]:
q6_result = spark.sql("""
SELECT Zone, COUNT(*)
FROM result
GROUP BY 1
ORDER BY 2
""")

In [92]:
q6_result.show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
|        Battery Park|      15|
|Saint Michaels Ce...|      23|
|Breezy Point/Fort...|      25|
|Marine Park/Floyd...|      26|
|        Astoria Park|      29|
|    Inwood Hill Park|      39|
|       Willets Point|      47|
|Forest Park/Highl...|      53|
|  Brooklyn Navy Yard|      57|
|        Crotona Park|      62|
|        Country Club|      77|
|     Freshkills Park|      89|
|       Prospect Park|      98|
|     Columbia Street|     105|
|  South Williamsburg|     110|
+--------------------+--------+
only showing top 20 rows

